# Database Debug Notebook
This notebook helps inspect the PostgreSQL database to debug the foreign key violation issue.

In [ ]:
import sys
sys.path.insert(0, '/Users/ferdi/Documents/agent-copilot/src')

import psycopg2
import psycopg2.extras
from urllib.parse import urlparse
import pandas as pd
from datetime import datetime
import os

# Import the database module
from aco.server import db

## Check Database Connection

In [ ]:
# Get the database URL from environment or config
from aco.common.constants import DATABASE_URL

if DATABASE_URL:
    print(f"Database URL found: {DATABASE_URL.split('@')[1] if '@' in DATABASE_URL else DATABASE_URL}")
    conn = db.get_conn()
    print("Connected to database successfully")
else:
    print("No DATABASE_URL found")

## List All Experiments

In [ ]:
# Get all experiments
experiments = db.query_all(
    "SELECT session_id, parent_session_id, name, timestamp, success, notes FROM experiments ORDER BY timestamp DESC LIMIT 20"
)

if experiments:
    df_experiments = pd.DataFrame(experiments)
    print(f"Found {len(experiments)} experiments:")
    display(df_experiments)
else:
    print("No experiments found in database")

## Check for Specific Session ID

In [ ]:
# Check for the problematic session_id from the error
problematic_session_id = "4c1ab5d2-5e84-402b-bd87-8f02250fe2fc"

exists = db.query_one(
    "SELECT * FROM experiments WHERE session_id = ?",
    (problematic_session_id,)
)

if exists:
    print(f"Session {problematic_session_id} EXISTS in experiments table:")
    for key, value in dict(exists).items():
        if key not in ['graph_topology', 'environment', 'log']:
            print(f"  {key}: {value}")
else:
    print(f"Session {problematic_session_id} DOES NOT EXIST in experiments table")

## Check LLM Calls Table

In [ ]:
# Get recent LLM calls
llm_calls = db.query_all(
    "SELECT session_id, node_id, api_type, timestamp FROM llm_calls ORDER BY timestamp DESC LIMIT 20"
)

if llm_calls:
    df_llm = pd.DataFrame(llm_calls)
    print(f"Found {len(llm_calls)} recent LLM calls:")
    display(df_llm)
else:
    print("No LLM calls found")

## Check for Orphaned LLM Calls

In [ ]:
# Find LLM calls with session_ids that don't exist in experiments
orphaned = db.query_all(
    """
    SELECT DISTINCT l.session_id, COUNT(*) as call_count
    FROM llm_calls l
    LEFT JOIN experiments e ON l.session_id = e.session_id
    WHERE e.session_id IS NULL
    GROUP BY l.session_id
    """
)

if orphaned:
    print(f"Found {len(orphaned)} session(s) with orphaned LLM calls:")
    for row in orphaned:
        print(f"  Session {row['session_id']}: {row['call_count']} orphaned calls")
else:
    print("No orphaned LLM calls found")

## Check Foreign Key Constraints

In [ ]:
# Check foreign key constraints in the database
constraints = db.query_all(
    """
    SELECT 
        tc.table_name, 
        kcu.column_name, 
        ccu.table_name AS foreign_table_name,
        ccu.column_name AS foreign_column_name 
    FROM 
        information_schema.table_constraints AS tc 
        JOIN information_schema.key_column_usage AS kcu
          ON tc.constraint_name = kcu.constraint_name
          AND tc.table_schema = kcu.table_schema
        JOIN information_schema.constraint_column_usage AS ccu
          ON ccu.constraint_name = tc.constraint_name
          AND ccu.table_schema = tc.table_schema
    WHERE tc.constraint_type = 'FOREIGN KEY'
    """
)

if constraints:
    print("Foreign key constraints:")
    for c in constraints:
        print(f"  {c['table_name']}.{c['column_name']} -> {c['foreign_table_name']}.{c['foreign_column_name']}")
else:
    print("No foreign key constraints found")

## Clean Up Orphaned Records (Optional)

In [ ]:
# Uncomment to delete orphaned LLM calls
# WARNING: This will delete data!

# orphaned_sessions = db.query_all(
#     """
#     SELECT DISTINCT l.session_id
#     FROM llm_calls l
#     LEFT JOIN experiments e ON l.session_id = e.session_id
#     WHERE e.session_id IS NULL
#     """
# )

# if orphaned_sessions:
#     for row in orphaned_sessions:
#         session_id = row['session_id']
#         print(f"Deleting orphaned LLM calls for session {session_id}")
#         db.execute("DELETE FROM llm_calls WHERE session_id = ?", (session_id,))
#     print(f"Deleted orphaned calls for {len(orphaned_sessions)} sessions")
# else:
#     print("No orphaned records to clean up")

## Insert Missing Experiment (Emergency Fix)

In [ ]:
# Create a placeholder experiment for the problematic session if it doesn't exist
# Uncomment and modify session_id to use

# import json
# from datetime import datetime

# session_to_fix = "4c1ab5d2-5e84-402b-bd87-8f02250fe2fc"

# exists = db.query_one(
#     "SELECT session_id FROM experiments WHERE session_id = ?",
#     (session_to_fix,)
# )

# if not exists:
#     print(f"Creating placeholder experiment for session {session_to_fix}")
#     db.execute(
#         "INSERT INTO experiments (session_id, parent_session_id, name, graph_topology, timestamp, cwd, command, environment, success, notes, log) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
#         (
#             session_to_fix,
#             session_to_fix,
#             "Recovered session",
#             json.dumps({"nodes": [], "edges": []}),
#             datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
#             "unknown",
#             "unknown",
#             json.dumps({}),
#             "",
#             "Auto-created to fix foreign key violation",
#             "",
#         ),
#     )
#     print("Experiment created successfully")
# else:
#     print(f"Session {session_to_fix} already exists")

## Monitor Real-time Insertions

In [ ]:
# Check the most recent experiments and LLM calls
import time

print("Most recent activity:")
print("\nLast 5 experiments:")
recent_exp = db.query_all(
    "SELECT session_id, name, timestamp FROM experiments ORDER BY timestamp DESC LIMIT 5"
)
for exp in recent_exp:
    print(f"  {exp['timestamp']}: {exp['session_id'][:8]}... - {exp['name']}")

print("\nLast 5 LLM calls:")
recent_llm = db.query_all(
    "SELECT session_id, node_id, timestamp FROM llm_calls ORDER BY timestamp DESC LIMIT 5"
)
for llm in recent_llm:
    print(f"  {llm['timestamp']}: {llm['session_id'][:8]}... - Node: {llm['node_id'][:8]}...")